## 05. 일원분산분석(one-way ANOVA)

피셔의 분산분석(ANOVA)이란?: 세 개 이상의 모집단에 대한 모평균의 차이에 대한 가설검정입니다. 회귀분석과 마찬가지로 p-값에 따라 가설의 유의수준을 파악합니다. 이어서 잔차가 정규성/등분산성을 만족하는지도 파악합니다. ANOVA 분석의 귀무가설은 ‘모든 집단의 평균이 동일하다’ 입니다. 따라서 p값이 0.05보다 작다면 95% 신뢰구간에서 적어도 한 집단의 모평균이 다른 집단의 모평균과 다르다고 할 수 있습니다.

만약 ANOVA 결과 집단별 모평균에서 차이가 발생했을 경우 정확히 어떤 그룹과 어떤 그룹 사이에서 모평균 차이가 발생했는지 알아 볼 필요가 있습니다. 이를 사후분석(Post hoc)이라고 하는데, FWER(FamilyWise Error Rate, 여러 귀무가설 중 적어도 하나의 귀무가설에서 참이지만 기각하는 오류-1종오류-가 발생할 확률, 즉 차이가 발생했다고 거짓말을 할 확률)를 평가합니다. FWER가 높을수록 p값이 작겠지요?

R에 포함된 통계데이터를 활용하여 실제 분석을 진행해 보겠습니다.

In [1]:
data('PlantGrowth')
head(PlantGrowth)

,weight,group
,<dbl>,<fct>
1,4.17,ctrl
2,5.58,ctrl
3,5.18,ctrl
4,6.11,ctrl
5,4.50,ctrl
6,4.61,ctrl


R에 기본으로 들어있는 PlantGrowth 데이터셋은 식물에 대한 실험에서 통제변인(ctrl)과 두 개의 조작변인(trt1, trt2)을 가한 식물들의 무게와 적용된 변인을 기록한 것입니다.

In [2]:
levels(PlantGrowth$group)     #그룹에 몇 개의 수준이 있는지 확인
#'ctrl', 'trt1', 'trt2'
anova1 = aov(weight ~ group, data = PlantGrowth)
summary(anova1)

[1] "ctrl" "trt1" "trt2"

            Df Sum Sq Mean Sq F value Pr(>F)  
group        2  3.766  1.8832   4.846 0.0159 *
Residuals   27 10.492  0.3886                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Pr(>F)가 0.0159라서 약 99% 신뢰수준에서 적어도 한 집단의 모평균이 다른 집단의 모평균과 다르다고 할 수 있습니다.

In [3]:
shapiro.test(PlantGrowth$weight[PlantGrowth$group=='ctrl'])
shapiro.test(PlantGrowth$weight[PlantGrowth$group=='trt1'])
shapiro.test(PlantGrowth$weight[PlantGrowth$group=='trt2'])


	Shapiro-Wilk normality test

data:  PlantGrowth$weight[PlantGrowth$group == "ctrl"]
W = 0.95668, p-value = 0.7475



	Shapiro-Wilk normality test

data:  PlantGrowth$weight[PlantGrowth$group == "trt1"]
W = 0.93041, p-value = 0.4519



	Shapiro-Wilk normality test

data:  PlantGrowth$weight[PlantGrowth$group == "trt2"]
W = 0.94101, p-value = 0.5643


정규성 검정입니다. 세 그룹 모두 정규성 가정을 위반하지 않는다고 볼 수 있습니다. 따라서 등분산성의 경우 자료가 정규성 가정을 위반했던 회귀분석때의 예시와는 다르게 location이 mean인 레빈 검정 혹은 바틀렛 검정(bartlett.test)을 실시해야 합니다.

lawstat 패키지 설치 및 로드가 필요합니다.

In [4]:
install.packages('lawstat')
library(lawstat)

Installing package into 'C:/Users/82107/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'lawstat' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\82107\AppData\Local\Temp\Rtmpy0b8YI\downloaded_packages


In [5]:
bartlett.test(PlantGrowth$weight, PlantGrowth$group)
#바틀렛 검정
levene.test(PlantGrowth$weight, PlantGrowth$group, location='mean')
#레빈 검정


	Bartlett test of homogeneity of variances

data:  PlantGrowth$weight and PlantGrowth$group
Bartlett's K-squared = 2.8786, df = 2, p-value = 0.2371



	Classical Levene's test based on the absolute deviations from the mean
	( none not applied because the location is not set to median )

data:  PlantGrowth$weight
Test Statistic = 1.237, p-value = 0.3062


두 검정에서 모두 등분산성 가정이 유지됩니다. 만약 ANOVA 결과 집단별 모평균에서 차이가 발생했을 경우 정확히 어떤 그룹과 어떤 그룹 사이에서 모평균 차이가 발생했는지 알아 볼 필요가 있습니다. 이를 사후분석(Post hoc)이라고 하는데, FWER(FamilyWise Error Rate, 여러 귀무가설 중 적어도 하나의 귀무가설에서 참이지만 기각하는 오류-1종오류-가 발생할 확률, 즉 차이가 발생했다고 거짓말을 할 확률)를 평가합니다. FWER가 높을수록 p값이 작겠지요?

ANOVA 분석 결과 그룹간 모평균 차이가 있었기 때문에, 사후분석을 해봅니다.

DescTools 패키지 설치 및 로드가 필요합니다.

In [6]:
install.packages('DescTools')
library(DescTools)

Installing package into 'C:/Users/82107/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'DescTools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\82107\AppData\Local\Temp\Rtmpy0b8YI\downloaded_packages


In [7]:
PostHocTest(anova1, method="lsd")
PostHocTest(anova1, method="bonferroni")
PostHocTest(anova1, method="hsd")
PostHocTest(anova1, method="scheffe")


  Posthoc multiple comparisons of means : Fisher LSD 
    95% family-wise confidence level

$group
            diff      lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -0.94301261 0.2010126 0.1944    
trt2-ctrl  0.494 -0.07801261 1.0660126 0.0877 .  
trt2-trt1  0.865  0.29298739 1.4370126 0.0045 ** 

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



  Posthoc multiple comparisons of means : Bonferroni 
    95% family-wise confidence level

$group
            diff     lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -1.0825786 0.3405786 0.5832    
trt2-ctrl  0.494 -0.2175786 1.2055786 0.2630    
trt2-trt1  0.865  0.1534214 1.5765786 0.0134 *  

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



  Posthoc multiple comparisons of means : Tukey HSD 
    95% family-wise confidence level

$group
            diff     lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -1.0622161 0.3202161 0.3909    
trt2-ctrl  0.494 -0.1972161 1.1852161 0.1980    
trt2-trt1  0.865  0.1737839 1.5562161 0.0120 *  

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



  Posthoc multiple comparisons of means: Scheffe Test 
    95% family-wise confidence level

$group
            diff     lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -1.0930531 0.3510531 0.4241    
trt2-ctrl  0.494 -0.2280531 1.2160531 0.2265    
trt2-trt1  0.865  0.1429469 1.5870531 0.0163 *  

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


method 별 특징

- 피셔의 LSD(‘lsd’): FWER 높음.
유의수준 보정 없음.

- 봉페로니 교정(‘bonferroni’)
FWER 중간.
모든 집단을 짝지어 평가, p값이 3배.

- 투키의 HSD(‘hsd’)
FWER 중간.

- 셰페의 방법(‘scheffe’)
FWER 낮음. 보수적. 귀무가설이 거짓이어도 수용할 가능성 있음.(2종오류)

FWER의 기준을 5%(0.05)로 정했을 때, p값이 0.05보다 작은 trt1-trt2 사이의 모평균 차이만 95% 신뢰구간에서 유의미하다고 할 수 있겠습니다.